In [2]:
import tensorflow as tf
from tensorflow import keras

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [3]:
model = keras.models.load_model('enc10_3.keras')

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 1)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 32)   320         ['input_1[0][0]']                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 32, 32, 32)   9248        ['conv2d[0][0]']                 
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 16, 16, 32)   0           ['conv2d_1[0][0]']               
                                                                                              

In [4]:
from tensorflow.keras.preprocessing import image_dataset_from_directory as im_ds_from_dir
image_dir='images/minis'
target_dir='images/minis'
target_size=(32,32) # set this to the target size you want
channels=1 # for color images
color_mode='grayscale'
seed=123
class_mode=None
batch_size=32 # set this to desired batch size
vsplit=.2 # set this to the validation split you want
image_gen=im_ds_from_dir(image_dir, color_mode=color_mode, batch_size=batch_size,image_size=target_size,seed=seed,subset='training',
                         validation_split=vsplit, labels=None, shuffle=False)
val_gen=im_ds_from_dir(image_dir, color_mode=color_mode, batch_size=batch_size,image_size=target_size,seed=seed,subset='validation',
                         validation_split=vsplit, labels=None, shuffle=False)
norm_layer = tf.keras.layers.Rescaling(1./255)
image_gen = image_gen.map(lambda x: (norm_layer(x) - 1e-6,norm_layer(x) - 1e-6))
val_gen = val_gen.map(lambda x: (norm_layer(x) - 1e-6,norm_layer(x) - 1e-6))

Found 182 files belonging to 1 classes.
Using 146 files for training.
Found 182 files belonging to 1 classes.
Using 36 files for validation.


In [5]:
cluster = keras.Model(
    inputs=model.inputs,
    outputs=model.get_layer(name='conv2d_7').output
)

bottleneck_features = cluster.predict(image_gen)

5/5 [==============================] - 1s 36ms/step


In [6]:
x = 0
for feature in bottleneck_features:
    x+=1

In [7]:
estimator = KMeans(n_clusters=3)

from sklearn.decomposition import PCA


estimator.fit(bottleneck_features.reshape(146,-1))

KMeans(n_clusters=3)

In [8]:
print(bottleneck_features.reshape(146,-1).shape)
print(bottleneck_features.flatten().shape)

(146, 4096)
(598016,)


In [9]:
labels = estimator.labels_

In [10]:
print(len(labels))

146


In [11]:
for data in image_gen:
    print(data)

(<tf.Tensor: shape=(32, 32, 32, 1), dtype=float32, numpy=
array([[[[0.75294024],
         [0.7607834 ],
         [0.7568618 ],
         ...,
         [0.8352932 ],
         [0.8431363 ],
         [0.8470579 ]],

        [[0.75294024],
         [0.7568618 ],
         [0.7568618 ],
         ...,
         [0.83921474],
         [0.8431363 ],
         [0.8470579 ]],

        [[0.7568618 ],
         [0.7568618 ],
         [0.7568618 ],
         ...,
         [0.8352932 ],
         [0.8431363 ],
         [0.8470579 ]],

        ...,

        [[0.8431363 ],
         [0.8431363 ],
         [0.8352932 ],
         ...,
         [0.91372454],
         [0.9058814 ],
         [0.9058814 ]],

        [[0.85097945],
         [0.8470579 ],
         [0.8313716 ],
         ...,
         [0.9058814 ],
         [0.909803  ],
         [0.909803  ]],

        [[0.85097945],
         [0.8431363 ],
         [0.82745004],
         ...,
         [0.90195984],
         [0.91372454],
         [0.909803  ]]],


  

In [14]:
from tensorflow.keras.preprocessing import image_dataset_from_directory as im_ds_from_dir
image_dir='images/minis'
target_dir='images/minis'
target_size=(32,32) # set this to the target size you want
channels=1 # for color images
color_mode='grayscale'
seed=123
class_mode=None
batch_size=32 # set this to desired batch size
vsplit=.2 # set this to the validation split you want
image_gen=im_ds_from_dir(image_dir, color_mode=color_mode, batch_size=batch_size,image_size=target_size,seed=seed,
                         labels=None, shuffle=False)

norm_layer = tf.keras.layers.Rescaling(1./255)
image_gen_map = image_gen.map(lambda x: (norm_layer(x) - 1e-6,norm_layer(x) - 1e-6))


Found 182 files belonging to 1 classes.


In [17]:
len(image_gen.file_paths) #ONLY WORKS WHEN NOT CALLED ON MAP OBJECT && SHUFFLE IS FALSE

182

from active_thresh_segseg import segseg
from microscope_image_preprocessing import load_image
for path in load_image('2024-07-29'):
    segseg(path,32)